# API Documentation Generation

> Generate module overviews with formatted signatures for nbdev projects

In [ ]:
#| default_exp api_docs

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
from pathlib import Path
from typing import List, Dict, Any, Optional
from nbdev.config import get_config
from cjm_nbdev_overview.core import *
from cjm_nbdev_overview.parsers import *
from cjm_nbdev_overview.tree import *
from dataclasses import dataclass
import textwrap

from execnb.nbio import read_nb, write_nb, mk_cell, new_nb
from fastcore.xtras import Path
import re

## Formatting Functions

In [ ]:
#| export
def format_function_doc(func: FunctionInfo,             # Function information
                       indent: str = ""                 # Indentation prefix
                       ) -> str:                        # Formatted documentation
    "Format a function with its signature for documentation"
    lines = []
    
    # Add the code block with signature
    lines.append("```python")
    
    # Add decorators if any
    for decorator in func.decorators:
        lines.append(f"{indent}@{decorator}")
    
    # Add the signature
    sig_lines = func.signature.split('\n')
    for line in sig_lines:
        lines.append(f"{indent}{line}")
    
    # Add docstring if present
    if func.docstring:
        docstring_lines = func.docstring.strip().split('\n')
        if len(docstring_lines) == 1:
            lines.append(f'{indent}    "{docstring_lines[0]}"')
        else:
            lines.append(f'{indent}    """')
            for doc_line in docstring_lines:
                lines.append(f'{indent}    {doc_line}')
            lines.append(f'{indent}    """')
    
    lines.append("```")
    
    return '\n'.join(lines)

In [ ]:
#| export
def format_class_doc(cls: ClassInfo                     # Class information
                    ) -> str:                           # Formatted documentation
    "Format a class with its signature and methods for documentation"
    lines = []
    
    lines.append("```python")
    
    # Add decorators if any
    for decorator in cls.decorators:
        lines.append(f"@{decorator}")
    
    # Add class signature
    lines.append(cls.signature)
    
    # Add class docstring if present
    if cls.docstring:
        docstring_lines = cls.docstring.strip().split('\n')
        if len(docstring_lines) == 1:
            lines.append(f'    "{docstring_lines[0]}"')
        else:
            lines.append('    """')
            for doc_line in docstring_lines:
                lines.append(f'    {doc_line}')
            lines.append('    """')
    
    # Add selected methods
    if cls.methods:
        lines.append("    ")
        # Show __init__ and public methods only
        shown_methods = [m for m in cls.methods if not m.name.startswith('_') or m.name == '__init__']
        
        for i, method in enumerate(shown_methods[:5]):  # Limit to first 5 methods
            if i > 0:
                lines.append("    ")
            
            # Format method signature
            method_lines = method.signature.split('\n')
            for j, line in enumerate(method_lines):
                # Remove the 'def' from the first line if it's there
                if j == 0 and line.strip().startswith('def'):
                    lines.append(f"    {line.strip()}")
                else:
                    lines.append(f"    {line}")
            
            # Add method docstring
            if method.docstring:
                lines.append(f'        "{method.docstring.strip()}"')
        
        if len(shown_methods) > 5:
            lines.append("        ...")
    
    lines.append("```")
    
    return '\n'.join(lines)

In [ ]:
#| export  
def format_variable_doc(var: VariableInfo               # Variable information
                       ) -> str:                        # Formatted documentation
    "Format a variable for documentation"
    parts = [var.name]
    
    if var.type_hint:
        parts.append(f": {var.type_hint}")
    
    if var.value:
        parts.append(f" = {var.value}")
    
    if var.comment:
        parts.append(f"  # {var.comment}")
    
    return ''.join(parts)

## Module Documentation Generation

In [ ]:
#| export
def generate_module_overview(module: ModuleInfo,        # Module information
                           show_all: bool = False       # Show all items including private
                           ) -> str:                    # Module overview markdown
    "Generate a markdown overview for a module"
    lines = []
    
    # Add module title
    if module.title:
        lines.append(f"### {module.title} (`{module.path.name}`)")
    else:
        lines.append(f"### {module.name} (`{module.path.name}`)")
    
    # Add description
    if module.description:
        lines.append(f"> {module.description}")
    
    # Filter items based on show_all and is_exported
    if show_all:
        functions = module.functions
        classes = module.classes  
        variables = module.variables
    else:
        functions = [f for f in module.functions if f.is_exported]
        classes = [c for c in module.classes if c.is_exported]
        variables = [v for v in module.variables if v.is_exported]
    
    # Add functions section
    if functions:
        lines.append("\n#### Functions\n")
        for func in functions:
            lines.append(format_function_doc(func))
            lines.append("")
    
    # Add classes section
    if classes:
        lines.append("#### Classes\n")
        for cls in classes:
            lines.append(format_class_doc(cls))
            lines.append("")
    
    # Add variables section
    if variables:
        lines.append("#### Variables\n")
        lines.append("```python")
        for var in variables:
            lines.append(format_variable_doc(var))
        lines.append("```")
    
    return '\n'.join(lines)

In [ ]:
#| export
def generate_project_api_docs(path: Path = None,        # Project path (defaults to nbs_path)
                            show_all: bool = False      # Show all items including private
                            ) -> str:                   # Full API documentation
    "Generate API documentation for all modules in a project"
    if path is None:
        cfg = get_config()
        path = cfg.nbs_path
    
    lines = []
    lines.append("## API Reference\n")
    
    # Get all notebooks
    notebooks = get_notebook_files(path, recursive=True)
    
    # Parse and document each notebook
    for nb_path in notebooks:
        # Skip index notebooks
        if nb_path.stem in ['index', '00_index']:
            continue
            
        try:
            module_info = parse_notebook(nb_path)
            
            # Only include if it has exported content
            has_exports = any([
                any(f.is_exported for f in module_info.functions),
                any(c.is_exported for c in module_info.classes),
                any(v.is_exported for v in module_info.variables)
            ])
            
            if has_exports or show_all:
                lines.append(generate_module_overview(module_info, show_all))
                lines.append("\n---\n")
        except Exception as e:
            print(f"Error parsing {nb_path}: {e}")
            continue
    
    return '\n'.join(lines)

## Index Update Function

The `update_index_module_docs` function can automatically update your project's index.ipynb with module documentation:

In [ ]:
#| export
def update_index_module_docs(index_path: Path = None,   # Path to index.ipynb (defaults to nbs/index.ipynb)
                           start_marker: str = "## Module Overview",  # Marker to identify module docs section
                           end_marker: str = "## ",     # Next section marker
                           ) -> None:                    # Updates index.ipynb in place
    "Update the module documentation section in index.ipynb"
    if index_path is None:
        cfg = get_config()
        index_path = cfg.nbs_path / "index.ipynb"
    
    # Read the existing notebook
    nb = read_nb(index_path)
    
    # Find or create the module overview section
    module_section_idx = None
    end_section_idx = None
    
    for i, cell in enumerate(nb.cells):
        if cell.cell_type == 'markdown':
            source = cell.source
            # Check if this is the module overview section
            if start_marker in source:
                module_section_idx = i
            # Check if we've reached the next section after module overview
            elif module_section_idx is not None and source.strip().startswith(end_marker):
                end_section_idx = i
                break
    
    # Get all notebooks and parse them
    notebooks = get_notebook_files(index_path.parent, recursive=True)
    module_cells = []
    
    # Create the module overview header cell
    header_cell = mk_cell(f"{start_marker}\n\nDetailed documentation for each module in the project:", 
                         cell_type='markdown')
    module_cells.append(header_cell)
    
    # Sort notebooks by their numeric prefix if they have one
    def sort_key(
        nb_path  # TODO: Add type hint and description
    ): # TODO: Add type hint
        "TODO: Add function description"
        match = re.match(r'^(\d+)', nb_path.stem)
        if match:
            return (int(match.group(1)), nb_path.stem)
        return (999, nb_path.stem)  # Put non-numbered notebooks at the end
    
    sorted_notebooks = sorted(notebooks, key=sort_key)
    
    # Generate overview for each module
    for nb_path in sorted_notebooks:
        # Skip index notebooks
        if nb_path.stem in ['index', '00_index']:
            continue
            
        try:
            module_info = parse_notebook(nb_path)
            
            # Only include if it has exported content
            has_exports = any([
                any(f.is_exported for f in module_info.functions),
                any(c.is_exported for c in module_info.classes),
                any(v.is_exported for v in module_info.variables)
            ])
            
            if has_exports:
                overview_md = generate_module_overview(module_info)
                overview_cell = mk_cell(overview_md, cell_type='markdown')
                module_cells.append(overview_cell)
        except Exception as e:
            print(f"Error parsing {nb_path}: {e}")
            continue
    
    # Now update the notebook
    if module_section_idx is not None:
        # Remove existing module documentation cells
        if end_section_idx is not None:
            # Remove cells from module_section_idx to end_section_idx (exclusive)
            del nb.cells[module_section_idx:end_section_idx]
        else:
            # Remove from module_section_idx to end of notebook
            del nb.cells[module_section_idx:]
        
        # Insert new module cells
        for i, cell in enumerate(module_cells):
            nb.cells.insert(module_section_idx + i, cell)
    else:
        # No module overview section found, append it
        for cell in module_cells:
            nb.cells.append(cell)
    
    # Write the updated notebook
    write_nb(nb, index_path)

In [ ]:
# Example: Update the index.ipynb with module documentation
# update_index_module_docs()  # This will update nbs/index.ipynb

# Or specify a custom path:
# update_index_module_docs(Path("../other_project/nbs/index.ipynb"))

## Testing

Let's test the API documentation generation on our project:

In [ ]:
# First, let's export our modules
import nbdev
nbdev.nbdev_export()

In [ ]:
# Test generating documentation for a single module
core_module = parse_notebook(Path("00_core.ipynb"))
print(generate_module_overview(core_module))

### Core Utilities (`00_core.ipynb`)
> Core utilities and data models for nbdev project overview generation

#### Functions

```python
def get_notebook_files(path: Path = None,           # Directory to search (defaults to nbs_path)
                      recursive: bool = True        # Search subdirectories
                      ) -> List[Path]:              # List of notebook paths
    "Get all notebook files in a directory"
```

```python
def get_subdirectories(path: Path = None,           # Directory to search (defaults to nbs_path)
                      recursive: bool = False       # Include all nested subdirectories
                      ) -> List[Path]:              # List of directory paths
    "Get subdirectories in a directory"
```

```python
def read_notebook(path: Path                    # Path to notebook file
                 ) -> Dict[str, Any]:           # Notebook content as dict
    "Read a notebook file and return its content"
```

```python
def get_cell_source(cell: 

In [ ]:
# Test full project API documentation (show first 1000 chars)
api_docs = generate_project_api_docs()
print(api_docs[:1000] + "..." if len(api_docs) > 1000 else api_docs)

## API Reference

### Core Utilities (`00_core.ipynb`)
> Core utilities and data models for nbdev project overview generation

#### Functions

```python
def get_notebook_files(path: Path = None,           # Directory to search (defaults to nbs_path)
                      recursive: bool = True        # Search subdirectories
                      ) -> List[Path]:              # List of notebook paths
    "Get all notebook files in a directory"
```

```python
def get_subdirectories(path: Path = None,           # Directory to search (defaults to nbs_path)
                      recursive: bool = False       # Include all nested subdirectories
                      ) -> List[Path]:              # List of directory paths
    "Get subdirectories in a directory"
```

```python
def read_notebook(path: Path                    # Path to notebook file
                 ) -> Dict[str, Any]:           # Notebook content as dict
    "Read a notebook file and return its content"
```

```python
def get_

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()